In [1]:
from dotenv import dotenv_values
import requests
import json
import random

config = dotenv_values("../.env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}
debug_server = config['DEBUG_SERVER']
token = config['TOKEN']
submitt_url = config['DEBUG_SERVER_SUBMIT']

## Official 

In [35]:
url = debug_server+token
post_url = submitt_url+token

In [36]:
post_url

'https://erisk.irlab.org/challenge-service/submit/gvq8q2KMn564w+0RADA15Hv3DfqPYXb2c6czyVxI5U8'

In [37]:
ans = requests.get(url, headers = {'content-type': 'application/json'})

In [38]:
ans_dict = json.loads(ans.text)

In [40]:
submission = []
for ans in ans_dict:
    nick = ans['nick']
    submission.append({
        'nick': nick,
        'decision': random.choice([0,1]),
        'score': round(random.uniform(0,4),1)
    })


In [41]:
x = requests.post(url, data = json.dumps(submission), headers = {'content-type': 'application/json'})

In [43]:
print(x)

<Response [405]>


## Local

In [37]:
test_url = 'http://127.0.0.1:5000'
ans = requests.get(test_url)
ans_dict = json.loads(ans.text)
print(ans)

<Response [200]>


In [39]:
submission = []
for ans in ans_dict:
    nick = ans['nick']
    text = ans['content']
    submission.append({
        'nick': nick,
        'decision': get_prediction(text),
        'score': round(random.uniform(0,4),1)
    })

In [40]:
x = requests.post(test_url, data = json.dumps(submission, indent=2), headers = {'content-type': 'application/json'})
print(x)

<Response [200]>


## Test implementation

In [33]:
def get_prediction(text):
    text = tokenizer.encode_plus(
    text,
    # add_special_tokens=True,
    max_length=512,
    padding="max_length",
    return_tensors="pt",
    return_attention_mask=False,
    truncation=True,
    )
    output = torch.sigmoid(model(text['input_ids'])).detach().numpy()
    return np.heaviside(output, 0)

In [32]:
from data import DataModule
from models import Transformer
from transformers import RobertaTokenizerFast
import torch
import numpy as np

model_path = r"../models/roberta_3/model_val_loss0.28.ckpt"
tokenizer_path =  r"../models/roberta-tokenizer"

In [12]:
model = Transformer(
    ntokens=30000,
    emsize=128,
    d_hid=128,
    nlayers=1,
    nhead=2,
    dropout=0.2,
).load_from_checkpoint(model_path)
model.eval()
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)


Transformer(
  (model): TransformerModel(
    (encoder): Embedding(30000, 128)
    (pos_encoder): PositionalEncoding(
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=128, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (decoder): Linear(in_features=128, out_features=1, bias=True)
  )
  (criterion): B